In [8]:
import re
from pathlib import Path

import pandas as pd

In [22]:
# download http://m3c.web.auth.gr/research/datasets/bdlib/

root = Path('/home/t9s9/Datasets/BDLib-2')
data_p = root / 'data' / 'test'
data_p.mkdir(parents=True, exist_ok=True)

result = []
for fold in root.iterdir():
    if fold.is_dir() and fold.name.startswith('fold'):
        fold_id = int(fold.name[-1]) - 1
        for file in fold.iterdir():
            if file.is_file():
                # use re to get the classname of the file, search for all letters until the first number
                classname = re.search(r'[a-zA-Z]*', file.name).group()
                result.append({'filename': file.name, 'classname': classname, 'fold': fold_id })
                file.rename(data_p / file.name)
    fold.rmdir()

df = pd.DataFrame(result)
df.classname = df.classname.astype('category')
df['class_id'] = df.classname.cat.codes
df['filename'] = df['filename'].apply(lambda x: str((Path('test') / x).with_suffix('.flac')))
df.to_parquet(root / 'annot_test.parquet', index=False)

In [32]:
df

,filename,classname,fold,class_id
0,test/airplane01.flac,airplane,0,0
1,test/applause02.flac,applause,0,2
2,test/thunders01.flac,thunders,0,9
3,test/thunders06.flac,thunders,0,9
4,test/rain06.flac,rain,0,6
...,...,...,...,...
175,test/airplane16.flac,airplane,2,0
176,test/motorcycles13.flac,motorcycles,2,5
177,test/rain13.flac,rain,2,6
178,test/seawaves13.flac,seawaves,2,8


In [35]:
idx_2_label = dict(enumerate(df.classname.cat.categories))
pd.Series(idx_2_label).rename('label').to_frame().to_parquet(root / 'idx_2_label.parquet')

In [37]:
idx_2_label.values()

dict_values(['airplane', 'alarms', 'applause', 'birds', 'dogs', 'motorcycles', 'rain', 'rivers', 'seawaves', 'thunders'])